In [1]:
import sys
import os
import fastText

from validation import compute_f1
import conlleval

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

import shutil, json

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

In [5]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [39]:
def load_model_indexes(indexes_file, max_sequence_length = 56):
    indexMappings = json.load(open(indexes_file, 'r', encoding='UTF-8'))
    models.idx2Label = {int(k):v for k,v in indexMappings[0].items()}
    models.label2Idx = indexMappings[1]
    models.char2Idx = indexMappings[2]
    models.case2Idx = indexMappings[3]
    models.max_sequence_length = max_sequence_length

# Test Model

## GermEval (outer)

In [21]:
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')
print(len(testSentences))

5100


In [22]:
load_model_indexes('../models/germeval.indexes')
finalmodel = load_model('../models/germeval.h5', custom_objects=create_custom_objects())

In [23]:
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

(0.8315377081292851, 0.8245386856587893, 0.8280234070221067)


## GermEval (inner)

In [24]:
idx_1st = models.idx2Label
print(models.label2Idx)
print(idx_1st)

{'I-OTHpart': 7, 'B-PER': 10, 'B-LOC': 12, 'I-OTHderiv': 20, 'B-PERpart': 23, 'B-ORGderiv': 2, 'I-PER': 19, 'I-PERderiv': 8, 'I-LOCpart': 17, 'O': 9, 'B-OTHderiv': 6, 'I-OTH': 4, 'I-ORGpart': 5, 'B-OTHpart': 24, 'I-PERpart': 13, 'I-LOC': 1, 'B-ORG': 18, 'B-OTH': 25, 'I-LOCderiv': 22, 'B-PERderiv': 11, 'B-LOCpart': 16, 'I-ORGderiv': 3, 'I-ORG': 15, 'B-ORGpart': 14, 'PADDING_TOKEN': 0, 'B-LOCderiv': 21}
{0: 'PADDING_TOKEN', 1: 'I-LOC', 2: 'B-ORGderiv', 3: 'I-ORGderiv', 4: 'I-OTH', 5: 'I-ORGpart', 6: 'B-OTHderiv', 7: 'I-OTHpart', 8: 'I-PERderiv', 9: 'O', 10: 'B-PER', 11: 'B-PERderiv', 12: 'B-LOC', 13: 'I-PERpart', 14: 'B-ORGpart', 15: 'I-ORG', 16: 'B-LOCpart', 17: 'I-LOCpart', 18: 'B-ORG', 19: 'I-PER', 20: 'I-OTHderiv', 21: 'B-LOCderiv', 22: 'I-LOCderiv', 23: 'B-PERpart', 24: 'B-OTHpart', 25: 'B-OTH'}


In [29]:
testSentences_2nd = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv', level2=True)
load_model_indexes('../models/germeval-inner.indexes')
finalmodel_2nd = load_model('../models/germeval-inner.h5', custom_objects=create_custom_objects())
true_labels_2nd, pred_labels_2nd = utils.predict_sequences(finalmodel_2nd, testSentences_2nd)
print(compute_f1(pred_labels_2nd, true_labels_2nd, models.idx2Label))
idx_2nd = models.idx2Label

(0.5938864628820961, 0.5281553398058253, 0.5590955806783146)


In [30]:
eval_file = 'germeval_outer_inner.tsv'
conlleval.write_germeval_file(true_labels, pred_labels, true_labels_2nd, pred_labels_2nd, testSentences, idx_1st, idx_2nd, eval_file) 

# Official script eval CoNLL

In [31]:
testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')
print(len(testSentences))

3005


In [38]:
print(testSentences[3])

[['Die', 'O'], ['sechs', 'O'], ['Abgeordneten', 'O'], ['der', 'O'], ['Grünen', 'B-ORG'], [',', 'O'], ['die', 'O'], ['seit', 'O'], ['Bildung', 'O'], ['einer', 'O'], ['Großen', 'O'], ['Koalition', 'O'], ['von', 'O'], ['Christ-', 'O'], ['und', 'O'], ['Sozialdemokraten', 'O'], ['die', 'O'], ['Opposition', 'O'], ['darstellen', 'O'], [',', 'O'], ['hatten', 'O'], ['sich', 'O'], ['beim', 'O'], ['Vorsteher', 'O'], ['Rainer', 'B-PER'], ['Bergert', 'I-PER'], ['für', 'O'], ['ihr', 'O'], ['Fehlen', 'O'], ['entschuldigen', 'O'], ['lassen', 'O'], ['.', 'O']]


In [40]:
load_model_indexes('../models/conll.indexes', max_sequence_length = 100)
finalmodel = load_model('../models/conll.h5', custom_objects=create_custom_objects())

In [41]:
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

(0.8708474576271187, 0.8422950819672131, 0.8563333333333334)


In [44]:
import conlleval

eval_file = 'test_pl.tsv'
conlleval.write_conll_file(true_labels, pred_labels, testSentences, models.idx2Label, eval_file) 
# p,r,f = conlleval.evaluate_conll_file(eval_file) 

In [ ]:
print(p)
print(r)
print(f)

## Averages

In [ ]:
import numpy as np

In [ ]:
file_base_name = 'model_lstm_germeval_v2.'
eval_file = 'test_pl.tsv'
all_f1_scores = []

for run_i in range(10):
    model_file = file_base_name + str(run_i) + '.h5'
    index_file = file_base_name + str(run_i) + '.h5.indexes'
    if not os.path.isfile(model_file):
        print(model_file + ' not found')
        continue
    if not os.path.isfile(index_file):
        print(model_file + ' not found')
        continue
    
    print('Evaluating model ' + model_file)
    
    load_model_indexes(index_file)
    finalmodel = load_model(model_file, custom_objects=create_custom_objects())
    
    true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
    
    conlleval.write_conll_file(true_labels, pred_labels, testSentences, models.idx2Label, eval_file) 
    p,r,f = conlleval.evaluate_conll_file(eval_file) 
    all_f1_scores.append(f)

print(all_f1_scores)
print(np.mean(all_f1_scores))

In [ ]:
# import importlib
# importlib.reload(conlleval)
# importlib.reload(utils)